In [1]:
from Scripts.Extract import ImportDataFromExcel, ImportDataFromPostgreSQL
from Scripts.Transform import CreateDimension, SplitByDelimiter, RenameColumns, CreateFactTable
from Scripts.Load import InsertDimToPostgreSQL, InsertFactToPostgreSQL

In [2]:
# Variables que usaremos

filepath = './Atenciones.xlsx'

nombre_columnas = {
                    'Numero Ticket': 'TicketID',
                    'Estado Ticket': 'Estado',
                    'Tipo Atencion': 'TipoAtencion',
                    'Fecha Creacion': 'FechaCreacion',
                    'Fecha Cierre': 'FechaCierre',
                    'Tipo': 'TipoItem',
                    'Tipo Proveedor': 'TipoProveedor'
                }

# 1. Extract
Atenciones = ImportDataFromExcel(filepath = filepath, sheet = 'Hoja1')

In [3]:
# 2. Renombrando

Atenciones = RenameColumns(Atenciones, nombre_columnas)
Atenciones = SplitByDelimiter(Atenciones, 'Ubicacion')
Atenciones = RenameColumns(Atenciones, {"Columna1": "AgenciaID", "Columna2": "Agencia"})
Atenciones = Atenciones.sort('FechaCreacion', descending=False) # Ordenar por FechaCreacion

In [5]:
# 3. Creando los DataFrames

Agencias = CreateDimension(Atenciones, 'AgenciaID','Agencia','Direccion','Distrito','Provincia','Departamento')
Estado = CreateDimension(Atenciones, 'Estado')
TipoAtencion = CreateDimension(Atenciones, 'TipoAtencion')
Proveedor = CreateDimension(Atenciones, 'Proveedor', 'TipoProveedor')
Item = CreateDimension(Atenciones, 'ItemID', 'Item', 'TipoItem', 'Categoria')

In [6]:
# 4. Probando Conexion

InsertDimToPostgreSQL(Agencias, "dimAgencia")
InsertDimToPostgreSQL(Item, "dimItem")
InsertDimToPostgreSQL(Proveedor, "dimProveedor")
InsertDimToPostgreSQL(TipoAtencion, "dimTipoAtencion")
InsertDimToPostgreSQL(Estado, "dimEstado")

In [7]:
# Extraer dimensiones con clave subrogada
# 5. Llamando al código

Proveedor = ImportDataFromPostgreSQL("dimProveedor", ['ProveedorID', 'Proveedor'])

TipoAtencion = ImportDataFromPostgreSQL("dimTipoAtencion", ['TipoAtencionID', 'TipoAtencion'])

Estado = ImportDataFromPostgreSQL("dimEstado", ['EstadoID', 'Estado'])

In [8]:
# 6. Haciendo el Join
Atenciones = CreateFactTable(Atenciones, Proveedor, Estado, TipoAtencion)

In [9]:
# 7. Escribiendo datos
InsertFactToPostgreSQL(Atenciones, "FactAtenciones")